In [9]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
import pickle
from sklearn.metrics import precision_score, recall_score, f1_score

# Step 1: Data Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the MNIST dataset
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Extract specific classes (3, 4, 7)
def extract_classes(dataset, classes):
    indices = [i for i, (_, label) in enumerate(dataset) if label in classes]
    subset = Subset(dataset, indices)
    label_map = {old_label: new_label for new_label, old_label in enumerate(classes)}
    mapped_labels = [(dataset[i][0], label_map[dataset[i][1]]) for i in indices]
    return mapped_labels

selected_classes = [3, 4, 7]
train_subset = extract_classes(train_dataset, selected_classes)
test_subset = extract_classes(test_dataset, selected_classes)

# Function to save datasets in pickle format
def save_data_to_pickle(dataset, filename):
    data = {'images': [], 'labels': []}
    for img, label in dataset:
        data['images'].append(img.numpy())
        data['labels'].append(label)
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

# Save individual class datasets
save_data_to_pickle(train_subset, 'train_rem.pkl')
for cls in selected_classes:
    class_subset = extract_classes(train_dataset, [cls])
    save_data_to_pickle(class_subset, f'train_{cls}.pkl')

# Create DataLoaders
def create_dataloader(dataset, batch_size=64, shuffle=True):
    images, labels = zip(*dataset)
    images = torch.stack(images)
    labels = torch.tensor(labels)
    return DataLoader(list(zip(images, labels)), batch_size=batch_size, shuffle=shuffle)

train_loader = create_dataloader(train_subset)
test_loader = create_dataloader(test_subset)

# Step 2: Model Definition
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(selected_classes))

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleNN()

# Step 3: Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Function
def train_model(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, epochs=5)
torch.save(model.state_dict(), 'mnist_model.pth')

# Evaluation Function
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total * 100
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

# Evaluate the model
evaluate_model(model, test_loader)

# Compute Task Vector
def compute_task_vector(model):
    return model.fc3.weight.detach().cpu().numpy()

# Save task vectors for individual classes
task_vectors = {}
for cls in selected_classes:
    class_subset = extract_classes(train_dataset, [cls])
    loader = create_dataloader(class_subset)
    train_model(model, loader, criterion, optimizer, epochs=5)
    task_vectors[cls] = compute_task_vector(model)
    pickle.dump(task_vectors[cls], open(f'tv_{cls}.pkl', 'wb'))

# Apply Combined Task Vector
def apply_combined_task_vector(model, task_vectors):
    combined_vector = sum(task_vectors.values()) / len(task_vectors)
    combined_vector /= torch.norm(torch.tensor(combined_vector))
    model.fc3.weight.data = torch.tensor(combined_vector, dtype=torch.float32)
    return model

# Apply the combined task vector and evaluate
apply_combined_task_vector(model, task_vectors)
evaluate_model(model, test_loader)

# Fine-tune with the combined task vector
train_model(model, train_loader, criterion, optimizer, epochs=3)
evaluate_model(model, test_loader)

Epoch 1/5, Loss: 0.1352
Epoch 2/5, Loss: 0.0678
Epoch 3/5, Loss: 0.0496
Epoch 4/5, Loss: 0.0423
Epoch 5/5, Loss: 0.0300
Accuracy: 99.07%
Precision: 0.99
Recall: 0.99
F1 Score: 0.99
Epoch 1/5, Loss: 0.0013
Epoch 2/5, Loss: 0.0001
Epoch 3/5, Loss: 0.0000
Epoch 4/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 1/5, Loss: 0.0128
Epoch 2/5, Loss: 0.0000
Epoch 3/5, Loss: 0.0000
Epoch 4/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Epoch 1/5, Loss: 0.0000
Epoch 2/5, Loss: 0.0000
Epoch 3/5, Loss: 0.0000
Epoch 4/5, Loss: 0.0000
Epoch 5/5, Loss: 0.0000
Accuracy: 33.44%
Precision: 0.78
Recall: 0.33
F1 Score: 0.17


/var/folders/j0/w_sc6_7n777fy7v8ytm1r08w0000gn/T/ipykernel_38388/2486548706.py:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.fc3.weight.data = torch.tensor(combined_vector, dtype=torch.float32)


Epoch 1/3, Loss: 0.3683
Epoch 2/3, Loss: 0.0329
Epoch 3/3, Loss: 0.0236
Accuracy: 98.97%
Precision: 0.99
Recall: 0.99
F1 Score: 0.99
